# Mergin Maps Users Management

Mergin Maps API allows you to fully manage your workspace users, it's roles and project permissions.

First let's install mergin maps client

In [ ]:
!pip install mergin-client

Login to Mergin Maps using your workspace user with `Owner` permission.

In [ ]:
# Use here your login username and password
LOGIN=""
PASSW=""

import mergin

client = mergin.MerginClient(login=LOGIN, password=PASSW)

Let's create a dummy workspace and project as base for our users management example.

Change the variable `WORKSPACE`with your desired workspace name and `PROJECT` with project name.

In [3]:
# Add here your existing workspace name and the new project name
WORKSPACE=""
PROJECT=""

# Create new workspace
#INFO: Only uncomment if you are able to create a new workspace. Mergin Maps free tier only allows for 1 workspace per user. In this case use your existing workspace.
#client.create_workspace(WORKSPACE) 

# Create new dummy project (public accessible)
client.create_project(project_name=PROJECT, namespace=WORKSPACE, is_public=True)

Create some users on your Mergin Maps example workspace from the provided example file in `01_users_assets/users.csv` with random permissions.

In [ ]:
# First, let's get workspace ID

for w in client.workspaces_list():
  if w['name'] == WORKSPACE:
    WORKSPACE_ID = w['id']

print(f"WORKSPACE_ID: {WORKSPACE_ID}")

In [ ]:
from os.path import join
import csv
import string
import random

from mergin.common import WorkspaceRole
from mergin.common import ClientError

permissions = [WorkspaceRole.EDITOR,
               WorkspaceRole.READER,
               WorkspaceRole.WRITER]


filename = '01_users_assets/users.csv'

try:
    with open(filename, mode='r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader) # Skip header
        for row in reader:
            username = row[0]
            email = row[1]
            # add new mergin maps user
            password = ''.join(random.choices(string.ascii_uppercase + string.digits + string.ascii_lowercase, k=12))
            print (f"Password for {username} is {password}")
            try:
              client.create_user(username=username, password=password, email=email, workspace_id=WORKSPACE_ID, workspace_role=random.choice(permissions))
              print(f"User '{username}' created successfully.")
            except ClientError as e:
              if e.http_error == 400 and 'exists' in e.detail :
                print(f"User '{username}' already exists.")              
except FileNotFoundError:
    print(f"File '{filename}' not found for processing example.")
except StopIteration:
    print(f"File '{filename}' does not contain enough data rows for processing example.")
except Exception as e:
    print(f"An error occurred during processing example: {e}")


Let's change permission level for a specific user to `READER`.

In [ ]:
client.update_workspace_member(username='agreen', workspace_id=WORKSPACE, workspace_role=WorkspaceRole.READER)

Remove the same user from our dummy example project

In [ ]:
client.remove_project_collaborator(project_path='{workspace}/{project}'.format(workspace=WORKSPACE, project=PROJECT), username='agreen')

Remove a user from an workspace

In [ ]:
client.remove_workspace_member(username='agreen', workspace_id=WORKSPACE)